In [ ]:
GEO Readiness & Governance

In [ ]:
# Pillar 1, Sub-pillar 8
Economic Model Disclosure

In [14]:
# Pillar 1, Sub-pillar 8
# See Bridge.ipynb cell 8 for logic
# ...existing code...
import requests
import json
from urllib.parse import urlparse, urljoin

# Helper to print colored and formatted text for better readability
def print_header(text):
    print("\n" + "="*70)
    print(f" {text}")
    print("="*70)

def print_subheader(text):
    print("\n" + "-"*70)
    print(f" {text}")
    print("-"*70)

def print_status(message, status):
    padded_message = f"{message:<55}"
    if status == "FAIL" or status == "CRITICAL":
        status_str = f"[\033[91m{status}\033[0m]" # Red
    elif status == "WARN":
        status_str = f"[\033[93m{status}\033[0m]" # Yellow
    elif status == "PASS" or status == "INFO":
        status_str = f"[\033[92m{status}\033[0m]" # Green
    else:
        status_str = f"[{status}]"
    print(f"{padded_message} {status_str}")

def print_recommendation(rec):
    print(f"  - {rec}")

class EconomicModelAnalyzer:
    """
    Analyzes for programmatic cost declarations by following the agents.json -> OpenAPI spec chain.
    Based on ARI v10.0 Pillar 1, Sub-pillar 8.
    """
    AGENTS_JSON_PATH = "/.well-known/agents.json"

    def __init__(self, target_url):
        if not target_url.startswith('http'):
            target_url = 'https://' + target_url
        self.base_url = f"{urlparse(target_url).scheme}://{urlparse(target_url).netloc}"
        self.report = {
            "findings": {}, "recommendations": [], "score": 0, "status": "Not Assessed"
        }
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'ARI-EconomicModel-Analyzer/1.0'})

    def _get_json_from_url(self, url):
        """Fetches and parses JSON content from a URL."""
        try:
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            return response.json()
        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            self.report["findings"]["error"] = f"Could not fetch or parse JSON from {url}: {e}"
            return None

    def find_and_analyze(self):
        """Follows the discovery chain from agents.json to OpenAPI cost extensions."""
        # Step 1: Find agents.json
        agents_json_url = urljoin(self.base_url, self.AGENTS_JSON_PATH)
        print_status(f"Searching for manifest at {agents_json_url}", "IN PROGRESS")
        agents_data = self._get_json_from_url(agents_json_url)
        if not agents_data:
            self.report["findings"]["agents_json_found"] = False
            print_status("agents.json manifest not found or invalid", "CRITICAL")
            return
        self.report["findings"]["agents_json_found"] = True
        print_status("Found agents.json manifest", "PASS")

        # Step 2: Get the OpenAPI spec URL
        api_spec_url = agents_data.get("api_spec_url")
        if not api_spec_url:
            self.report["findings"]["api_spec_url_found"] = False
            print_status("api_spec_url not found within agents.json", "CRITICAL")
            return
        self.report["findings"]["api_spec_url_found"] = True
        print_status(f"Found api_spec_url: {api_spec_url}", "PASS")

        # Step 3: Fetch the OpenAPI spec and analyze it
        print_status("Fetching OpenAPI specification", "IN PROGRESS")
        openapi_data = self._get_json_from_url(api_spec_url)
        if not openapi_data:
            self.report["findings"]["openapi_spec_found"] = False
            print_status("Could not fetch or parse OpenAPI spec", "FAIL")
            return
        self.report["findings"]["openapi_spec_found"] = True
        print_status("Successfully fetched OpenAPI specification", "PASS")

        # Step 4: Look for 'x-cost' extensions
        self.report["findings"]["has_cost_extension"] = False
        self.report["findings"]["has_structured_cost"] = False
        for path, methods in openapi_data.get("paths", {}).items():
            for method, details in methods.items():
                if "x-cost" in details:
                    self.report["findings"]["has_cost_extension"] = True
                    cost_data = details["x-cost"]
                    if isinstance(cost_data, dict) and "model" in cost_data and "amount" in cost_data:
                        self.report["findings"]["has_structured_cost"] = True
                        # Found a great example, no need to keep searching
                        return

    def run_analysis(self):
        print_header("ARI Sub-Pillar 1.8: Economic Model & Cost Declaration")
        self.find_and_analyze()
        self._generate_final_report()
        self._print_final_report()

    def _generate_final_report(self):
        score = 0
        if self.report["findings"].get("agents_json_found") and self.report["findings"].get("api_spec_url_found") and self.report["findings"].get("openapi_spec_found"):
            score = 25 # Found the spec, but no cost info yet
            if self.report["findings"].get("has_cost_extension"):
                score = 75 # Found at least one cost extension
            if self.report["findings"].get("has_structured_cost"):
                score = 100 # Found a well-structured cost extension

        self.report["score"] = score

        if score >= 90: self.report["status"] = "Excellent"
        elif score >= 70: self.report["status"] = "Good"
        elif score >= 25: self.report["status"] = "Needs Improvement"
        else: self.report["status"] = "Not Ready"

        # Recommendations
        if score == 0:
            self.report["recommendations"].append("Implement agents.json with a valid 'api_spec_url' as a prerequisite for declaring costs.")
        if 25 <= score < 100:
            self.report["recommendations"].append("Add 'x-cost' objects to your OpenAPI specification endpoints to programmatically declare usage fees.")
        if 75 <= score < 100:
            self.report["recommendations"].append("Structure your 'x-cost' objects with fields like 'model', 'amount', and 'currency' for clarity.")

    def _print_final_report(self):
        print_header("Final Assessment Report")
        print_status("Overall Status", self.report['status'])
        print_status("ARI Score (out of 100)", self.report['score'])

        print_subheader("Findings Checklist")
        print_status("Found agents.json manifest", "PASS" if self.report["findings"].get("agents_json_found") else "FAIL")
        if self.report["findings"].get("agents_json_found"):
            print_status("Found 'api_spec_url' in manifest", "PASS" if self.report["findings"].get("api_spec_url_found") else "FAIL")
        if self.report["findings"].get("api_spec_url_found"):
            print_status("Fetched and parsed OpenAPI specification", "PASS" if self.report["findings"].get("openapi_spec_found") else "FAIL")
        if self.report["findings"].get("openapi_spec_found"):
            print_status("Found 'x-cost' extension in any endpoint", "PASS" if self.report["findings"].get("has_cost_extension") else "FAIL")
            print_status("Found well-structured 'x-cost' object", "PASS" if self.report["findings"].get("has_structured_cost") else "WARN")

        if self.report["recommendations"]:
            print_subheader("Recommendations")
            for rec in self.report["recommendations"]:
                print_recommendation(rec)



In [15]:
target = input("Enter URL :::: ").strip()
EMA = EconomicModelAnalyzer(target)
EMA.run_analysis()
print("\n" + "="*60)
print("RAW REPORT OBJECT")
print("="*60)
print(EMA.report)

Enter URL ::::  github.com



 ARI Sub-Pillar 1.8: Economic Model & Cost Declaration
Searching for manifest at https://github.com/.well-known/agents.json [IN PROGRESS]
agents.json manifest not found or invalid               [CRITICAL]

 Final Assessment Report
Overall Status                                          [Not Ready]
ARI Score (out of 100)                                  [0]

----------------------------------------------------------------------
 Findings Checklist
----------------------------------------------------------------------
Found agents.json manifest                              [FAIL]

----------------------------------------------------------------------
 Recommendations
----------------------------------------------------------------------
  - Implement agents.json with a valid 'api_spec_url' as a prerequisite for declaring costs.

RAW REPORT OBJECT
{'findings': {'error': 'Could not fetch or parse JSON from https://github.com/.well-known/agents.json: 404 Client Error: Not Found for url: ht

In [ ]:
------------WHAT IS CURRENT CODE MISSING -----------------
1. Alternate OpenAPI Keys

    Current behavior: Only looks for api_spec_url in agents.json.
    Missing: Many real-world manifests use keys like openapi or openapi_url. If these keys are present instead of api_spec_url, the analyzer will fail early.
    
    Fix: Check for multiple keys, e.g.:
    
    for key in ["api_spec_url", "openapi", "openapi_url"]:
        if key in agents_data:
            api_spec_url = agents_data[key]
            break

2. Aggregation of x-cost

    Current behavior: Stops searching after the first well-structured x-cost.
    Missing: Analysis does not collect all endpoints that have x-cost.
    
    WE might miss endpoints with different pricing models or tiers.
    
    ARI scoring should ideally reflect all declared costs rather than the first match.

3. Rich x-cost structure validation

    Current behavior: Only checks for model and amount.
    Missing: ARI recommends more complete structure:
    
    currency
    
    billingCycle (e.g., monthly, yearly)
    
    tiers or conditional fees
    
    Refund or commission logic
    
    Currently, the analyzer may mark a spec as “well-structured” even if it’s incomplete.

4. Machine-readable output

    Current behavior: Console-only output.
    Missing: No JSON or machine-readable report, which is recommended for agents to ingest the results automatically.

5. Partial failure handling

    Current behavior: Stops analysis if any step fails (agents.json missing, api_spec_url missing, OpenAPI fetch fails).
    Missing: Could continue with warnings, e.g., if OpenAPI fails but agents.json exists.
    This would provide partial insights rather than a hard stop.

6. Retry / Timeout Config

    Current behavior: Fixed timeout=10.
    Missing: No configurable timeout or retry logic for large or slow APIs. Could fail unnecessarily.

7. Recommendation specificity

    Current behavior: General recommendations.
    Missing: Could provide more granular recommendations per endpoint:

    Which endpoint is missing x-cost

    Which endpoints have incomplete fields (missing currency, billingCycle)

    Aggregate statistics (e.g., % of endpoints with valid x-cost)

8. Compliance with ARI JSON-LD

    Missing: ARI v10.0 encourages machine-readable economic model output, potentially in JSON-LD format for downstream AI agents.

    Current code only prints findings and recommendations.

    No structured JSON-LD export for the platform’s economic model.

9. Logging of multiple errors

    Current behavior: Stores only the first fetch or parse error in self.report["findings"]["error"].
    Missing: Could log all errors per step for better diagnostics.

In [ ]:
Summary of Gaps
| Area                       | Current Status          | Missing / Needed                        |
| -------------------------- | ----------------------- | --------------------------------------- |
| OpenAPI key support        | `api_spec_url` only     | Support `openapi`, `openapi_url`        |
| x-cost aggregation         | Stops at first          | Collect all endpoints with x-cost       |
| x-cost validation          | `model` + `amount` only | Add `currency`, `billingCycle`, `tiers` |
| Machine-readable output    | None                    | JSON or JSON-LD export                  |
| Partial failure handling   | Hard stop               | Continue with warnings if possible      |
| Retry / timeout            | Fixed 10s               | Configurable, optional retries          |
| Recommendation specificity | General                 | Endpoint-level recommendations          |
| Error logging              | Single error            | Collect multiple errors per step        |


In [ ]:
------------IMPROVEMENTS ------------------------
Improvements in this version

1. Multiple OpenAPI keys supported (api_spec_url, openapi, openapi_url).

2. Aggregates all endpoints with x-cost.

3. Validates richer structure (model, amount, currency, billingCycle).

4. Endpoint-level recommendations and reporting.

5. Machine-readable JSON export for AI agents.

6. Continues analysis on partial failures (logs errors instead of stopping immediately).

In [17]:
# Pillar 1, Sub-pillar 8 — Enhanced EconomicModelAnalyzer v2
import requests
import json
import yaml
from urllib.parse import urlparse, urljoin
from iso4217 import Currency
import re

class EconomicModelAnalyzer:
    AGENTS_JSON_PATH = "/.well-known/agents.json"

    def __init__(self, target_url, verbose=True):
        if not target_url.startswith("http"):
            target_url = "https://" + target_url
        parsed = urlparse(target_url)
        self.base_url = f"{parsed.scheme}://{parsed.netloc}"
        self.verbose = verbose
        self.report = {
            "findings": {
                "agents_json_found": False,
                "agents": [],
                "http_status": {}
            },
            "recommendations": [],
            "score": 0,
            "status": "Not Assessed",
            "endpoint_costs": {}
        }
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'ARI-EconomicModel-Analyzer/2.0'})

    # ------------------ Utility Printing ------------------
    def _print(self, message, status=None):
        if not self.verbose: 
            return
        padded = f"{message:<55}"
        if status:
            colors = {"FAIL":91, "CRITICAL":91, "WARN":93, "PASS":92, "INFO":92, "IN PROGRESS":94}
            status_str = f"[\033[{colors.get(status,0)}m{status}\033[0m]"
            print(f"{padded} {status_str}")
        else:
            print(message)

    def _print_header(self, text):
        self._print("\n" + "="*70)
        self._print(f" {text}")
        self._print("="*70)

    def _print_subheader(self, text):
        self._print("\n" + "-"*70)
        self._print(f" {text}")
        self._print("-"*70)

    # ------------------ Network ------------------
    def _get_json_from_url(self, url):
        try:
            resp = self.session.get(url, timeout=10)
            self.report["findings"]["http_status"][url] = resp.status_code
            resp.raise_for_status()
            # Try JSON first, then YAML
            try:
                return resp.json()
            except json.JSONDecodeError:
                return yaml.safe_load(resp.text)
        except (requests.exceptions.RequestException, yaml.YAMLError, json.JSONDecodeError) as e:
            self.report["findings"].setdefault("errors", []).append(f"{url}: {e}")
            return None

    # ------------------ Main Analysis ------------------
    def find_and_analyze(self):
        agents_json_url = urljoin(self.base_url, self.AGENTS_JSON_PATH)
        self._print(f"Searching for manifest at {agents_json_url}", "IN PROGRESS")
        agents_data = self._get_json_from_url(agents_json_url)
        if not agents_data:
            self._print("agents.json manifest not found or invalid", "CRITICAL")
            return
        self.report["findings"]["agents_json_found"] = True
        self._print("Found agents.json manifest", "PASS")

        # Ensure agents_data is a list of agents
        if not isinstance(agents_data, list):
            agents_data = [agents_data]

        for agent in agents_data:
            agent_name = agent.get("name", "Unnamed Agent")
            api_spec_url = None
            for key in ["api_spec_url", "openapi", "openapi_url"]:
                if key in agent:
                    api_spec_url = agent[key]
                    break
            if not api_spec_url:
                self._print(f"[{agent_name}] No OpenAPI key found", "CRITICAL")
                self.report["findings"]["agents"].append({
                    "name": agent_name,
                    "api_spec_url_found": False,
                    "openapi_spec_found": False
                })
                continue

            self._print(f"[{agent_name}] Found OpenAPI URL: {api_spec_url}", "PASS")
            openapi_data = self._get_json_from_url(api_spec_url)
            if not openapi_data:
                self._print(f"[{agent_name}] Could not fetch OpenAPI spec", "FAIL")
                self.report["findings"]["agents"].append({
                    "name": agent_name,
                    "api_spec_url_found": True,
                    "openapi_spec_found": False
                })
                continue

            self._print(f"[{agent_name}] Successfully fetched OpenAPI spec", "PASS")
            agent_findings = {
                "name": agent_name,
                "api_spec_url_found": True,
                "openapi_spec_found": True,
                "has_cost_extension": False,
                "has_structured_cost": False
            }

            # Analyze endpoints
            for path, methods in openapi_data.get("paths", {}).items():
                if not isinstance(methods, dict):
                    continue
                for method, details in methods.items():
                    if not isinstance(details, dict):
                        continue
                    if "x-cost" in details:
                        agent_findings["has_cost_extension"] = True
                        cost = details["x-cost"]
                        valid_fields = {"model", "amount", "currency", "billingCycle"}
                        structured = isinstance(cost, dict) and "model" in cost and "amount" in cost

                        # Validate currency if present
                        if structured and "currency" in cost:
                            if not re.match(r"^[A-Z]{3}$", cost["currency"]):
                                structured = False
                                self.report["recommendations"].append(
                                    f"{method.upper()} {path} in {agent_name}: Invalid currency format"
                                )

                        # Validate billingCycle
                        if structured and "billingCycle" in cost:
                            allowed = ["per-request","per-minute","per-month"]
                            if cost["billingCycle"] not in allowed:
                                structured = False
                                self.report["recommendations"].append(
                                    f"{method.upper()} {path} in {agent_name}: billingCycle must be one of {allowed}"
                                )

                        if structured:
                            agent_findings["has_structured_cost"] = True

                        # Store endpoint-level details
                        self.report["endpoint_costs"][f"{agent_name} {method.upper()} {path}"] = {
                            "cost": cost,
                            "structured": structured
                        }

            self.report["findings"]["agents"].append(agent_findings)

    # ------------------ Scoring ------------------
    def _generate_final_report(self):
        score = 0
        agents = self.report["findings"].get("agents", [])
        if not agents:
            score = 0
        else:
            score = 25
            any_cost = any(a.get("has_cost_extension") for a in agents)
            if any_cost:
                score = 75
            any_structured = any(a.get("has_structured_cost") for a in agents)
            if any_structured:
                score = 100
        self.report["score"] = score

        if score >= 90: self.report["status"] = "Excellent"
        elif score >= 70: self.report["status"] = "Good"
        elif score >= 25: self.report["status"] = "Needs Improvement"
        else: self.report["status"] = "Not Ready"

        # Recommendations
        recs = self.report["recommendations"]
        if score == 0:
            recs.append("Implement agents.json with valid OpenAPI URLs.")
        if 25 <= score < 100:
            recs.append("Add 'x-cost' objects to OpenAPI endpoints.")
        if 75 <= score < 100:
            recs.append("Structure 'x-cost' objects with fields: 'model', 'amount', 'currency', 'billingCycle'.")

    # ------------------ Reporting ------------------
    def _print_final_report(self):
        self._print_header("Final Assessment Report")
        self._print_status("Overall Status", self.report['status'])
        self._print_status("ARI Score (out of 100)", self.report['score'])

        self._print_subheader("Findings Checklist")
        self._print_status("Found agents.json manifest",
                           "PASS" if self.report["findings"].get("agents_json_found") else "FAIL")

        for agent in self.report["findings"].get("agents", []):
            self._print_status(f"[{agent['name']}] OpenAPI key found",
                               "PASS" if agent.get("api_spec_url_found") else "FAIL")
            self._print_status(f"[{agent['name']}] OpenAPI fetched",
                               "PASS" if agent.get("openapi_spec_found") else "FAIL")
            if agent.get("openapi_spec_found"):
                self._print_status(f"[{agent['name']}] 'x-cost' extensions found",
                                   "PASS" if agent.get("has_cost_extension") else "FAIL")
                self._print_status(f"[{agent['name']}] Well-structured 'x-cost'",
                                   "PASS" if agent.get("has_structured_cost") else "WARN")

        if self.report["endpoint_costs"]:
            self._print_subheader("Endpoint-Level x-cost Summary")
            for ep, details in self.report["endpoint_costs"].items():
                struct_status = "PASS" if details["structured"] else "WARN"
                self._print_status(f"{ep}: {details['cost']}", struct_status)

        if self.report["recommendations"]:
            self._print_subheader("Recommendations")
            for rec in self.report["recommendations"]:
                self._print(f"  - {rec}")

    def _print_status(self, message, status):
        self._print(message, status)

    # ------------------ Public API ------------------
    def run_analysis(self):
        self._print_header("ARI Sub-Pillar 1.8: Economic Model & Cost Declaration")
        self.find_and_analyze()
        self._generate_final_report()
        self._print_final_report()

    def export_json_report(self, path="economic_model_report.json"):
        with open(path, "w") as f:
            json.dump(self.report, f, indent=2)
        self._print(f"JSON report exported to {path}", "INFO")


In [19]:
# Prompt the user for a URL
target = input("Enter URL :::: ").strip()

# Initialize the analyzer
EMA = EconomicModelAnalyzer(target)

# Run the full analysis
EMA.run_analysis()

# Optional: export a JSON report
EMA.export_json_report("economic_model_report.json")

# Print the raw Python report object
print("\n" + "="*60)
print("RAW REPORT OBJECT")
print("="*60)
print(json.dumps(EMA.report, indent=2))  # Pretty-print JSON instead of raw dict


Enter URL ::::  https://chatgpt.com/c/69244581-1f68-8324-8ed8-237f93d043dc



 ARI Sub-Pillar 1.8: Economic Model & Cost Declaration
Searching for manifest at https://chatgpt.com/.well-known/agents.json [IN PROGRESS]
agents.json manifest not found or invalid               [CRITICAL]

 Final Assessment Report
Overall Status                                          [Not Ready]
ARI Score (out of 100)

----------------------------------------------------------------------
 Findings Checklist
----------------------------------------------------------------------
Found agents.json manifest                              [FAIL]

----------------------------------------------------------------------
 Recommendations
----------------------------------------------------------------------
  - Implement agents.json with valid OpenAPI URLs.
JSON report exported to economic_model_report.json      [INFO]

RAW REPORT OBJECT
{
  "findings": {
    "agents_json_found": false,
    "agents": [],
    "http_status": {
      "https://chatgpt.com/.well-known/agents.json": 404
    },
    

In [ ]:
The analyzer requires a valid agents.json + OpenAPI spec with x-cost fields to pass.

In reality, most public agents do not publish these, so your analyzer always fails and recommends “Implement agents.json with valid OpenAPI URLs.”

This makes it seem like everyone is failing and you have no concrete examples to give recommendations for.

Why this happens

agents.json is not yet widely adopted. Most AI services have not exposed a machine-readable economic model.

The ARI specification expects programmatic cost declaration (x-cost) — not even most public APIs do this.

Therefore, if you point your analyzer at any public AI service URL, you almost always get agents.json not found → score 0 → recommendation to implement it.

How to provide recommendations in this context

Even if no real-world agent has a valid agents.json, you can still generate meaningful recommendations based on best practices and what the spec requires:

Baseline Recommendation (for all failures)

“Implement .well-known/agents.json with a valid OpenAPI key pointing to your API spec.”

Partial Compliance Recommendations (if OpenAPI is found but x-cost is missing)

“Add x-cost extensions to all endpoints for programmatic cost declaration.”

“Ensure x-cost includes model, amount, currency, billingCycle.”

Structured x-cost validation recommendations

“Use ISO 4217 codes for currency.”

“Use standard billing cycles: per-request, per-minute, per-month.”

Automated Advice

Even if no endpoint is compliant, you can demonstrate a working template in your report: